In [1]:

%matplotlib inline
import numpy as np
import pandas as pd
import re
import sklearn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report


In [2]:
data = pd.read_csv("evalution_data_set.csv")
data.head()
data = data.sample(frac=1).reset_index(drop=True)

In [3]:
def _entity_string_to_dict(entity_string):
    entity_extract_pattern = re.compile(r'(?P<entity>\[(?P<value>.+?)\]\((?P<name_and_synonyms>.+?)\))')
    new_string = ''
    start = 0
    output = dict()
    output['entities'] = []
    for item in re.finditer(entity_extract_pattern, entity_string):
        d = dict()
        new_string += entity_string[start:item.start()]
        start = item.start()
        d['span_start'] = len(new_string)
        new_string += item.group('value')
        d['span_end'] = len(new_string)
        start += len(item.group('entity'))
        d['entity_value'] = item.group('value')

        syn_items = item.group('name_and_synonyms').split('|')
        d['entity_type'] = syn_items[0]
        d['synonyms'] = list()
        if len(syn_items) > 1:
            d['synonyms'] += [t for t in syn_items[1:]]

        output['entities'].append(d)
    new_string += entity_string[start:]
    output['statement'] = new_string
    return output


data['statement'] = data.apply(lambda x: _entity_string_to_dict(x['expression'])['statement'], axis=1)
data['entities'] = data.apply(lambda x: _entity_string_to_dict(x['expression'])['entities'], axis=1)
data.head()

,label,expression,statement,entities
0,intent_flight_status,will you be able to check status on my flight ...,will you be able to check status on my flight ...,"[{'span_start': 53, 'span_end': 56, 'entity_va..."
1,intent_flight_status,please check status of my flight?,please check status of my flight?,[]
2,intent_flight_search,I want a flight from [Colombo](Departure.Airpo...,I want a flight from Colombo,"[{'span_start': 21, 'span_end': 28, 'entity_va..."
3,intent_flight_status,when does flight [214](FlightNumber|412|867|30...,when does flight 214 depart ?,"[{'span_start': 17, 'span_end': 20, 'entity_va..."
4,intent_flight_search,please help me to search for a flight,please help me to search for a flight,[]


In [4]:
def simple_split(data,y,length,split_mark=0.7):
    if split_mark > 0. and split_mark < 1.0:
        n = int(split_mark*length)
    else:
        n = int(split_mark)
    x_train = data[:n].copy()
    x_test = data[n:].copy()
    y_train = y[:n].copy()
    y_test = y[n:].copy()
    return x_train,x_test,y_train,y_test

In [5]:
vectorizer = CountVectorizer()
x_train,x_test,y_train,y_test = simple_split(data.statement,data.label,len(data))
print(x_train.shape,x_test.shape)

(54,) (24,)


In [6]:
data.groupby('label').count()

,expression,statement,entities
label,,,
intent_flight_search,25,25,25
intent_flight_status,28,28,28
intent_provide_flight_info,25,25,25


In [7]:
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [8]:
feature_names = vectorizer.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("Number of features: {}".format(feature_names[:80]))

Number of features: 92
Number of features: ['100', '12th', '176', '214', '234', '296', '400', '412', '417', '451', '456', '501', '562', '571', '581', '613', '688', '751', '762', '781', '944', '967', 'able', 'about', 'airport', 'airportcode', 'are', 'be', 'calgary', 'can', 'check', 'cmb', 'colombo', 'depart', 'departe', 'departing', 'departure', 'does', 'dubai', 'find', 'flight', 'flightnumber', 'flights', 'flying', 'fo', 'for', 'fran', 'francisco', 'fransisco', 'friday', 'from', 'going', 'heathrow', 'help', 'how', 'is', 'jfk', 'june', 'lax', 'leave', 'leaving', 'me', 'monday', 'my', 'need', 'next', 'number', 'of', 'on', 'ontime', 'our', 'planning', 'please', 'san', 'search', 'sfo', 'slc', 'status', 'sunday', 'tell']


In [9]:
vectorizer.vocabulary_

{'will': 89,
 'you': 90,
 'be': 27,
 'able': 22,
 'to': 82,
 'check': 30,
 'status': 77,
 'on': 68,
 'my': 63,
 'flight': 40,
 'number': 66,
 '571': 13,
 'leaving': 60,
 'today': 83,
 'from': 50,
 'jfk': 56,
 'airport': 24,
 'please': 72,
 'of': 67,
 'want': 85,
 'colombo': 32,
 'when': 88,
 'does': 37,
 '214': 3,
 'depart': 33,
 'help': 53,
 'me': 61,
 'search': 74,
 'for': 45,
 'can': 29,
 'find': 39,
 'we': 86,
 'were': 87,
 'flying': 43,
 '688': 16,
 'leave': 59,
 'slc': 76,
 'time': 81,
 'are': 26,
 '12th': 1,
 'june': 57,
 'dubai': 38,
 'yyz': 91,
 '412': 7,
 'flights': 42,
 'tomorrow': 84,
 'need': 64,
 'the': 80,
 'next': 65,
 '234': 4,
 'calgary': 28,
 '100': 0,
 'sfo': 75,
 'departing': 35,
 'planning': 71,
 'sunday': 78,
 'lax': 58,
 'is': 55,
 'ontime': 69,
 'cmb': 31,
 '456': 10,
 '781': 19,
 '944': 20,
 'departure': 36,
 'departe': 34,
 'fo': 44,
 '581': 14,
 '451': 9,
 'going': 51,
 '400': 6,
 '562': 12,
 '501': 11,
 'monday': 62,
 'tell': 79,
 '762': 18,
 'flightnumber'

In [10]:
scores = cross_val_score(LogisticRegression( max_iter=100),vectorizer.transform(data.statement),data.label,cv=5)
print("Mean cross validation accuracy: {:.2f}".format(np.mean(scores)))

Mean cross validation accuracy: 0.92


In [11]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train,y_train)
print("Training set score: {:.3f}".format(logreg.score(x_train,y_train)))
print("Testing set score: {:.3f}".format(logreg.score(x_test,y_test)))

Training set score: 1.000
Testing set score: 0.875


In [12]:
 
sorted_labels = sorted(
    data.label.unique(), 
    key=lambda name: (name[1:], name[0])
)

In [13]:
pred_logreg = logreg.predict(x_test)
confusion_logreg = confusion_matrix(y_test,pred_logreg)
print("Confusion matrix:\n ")

header = pd.MultiIndex.from_product([['Predicted label'],
                                     sorted_labels],names=['','Actual label'])
confusion_mtrix_df =  pd.DataFrame(confusion_logreg,index=sorted_labels, 
                  columns=header)
confusion_mtrix_df

Confusion matrix:
 


Predicted label                       \
Actual label               intent_flight_search intent_flight_status   
intent_flight_search                          9                    0   
intent_flight_status                          0                    7   
intent_provide_flight_info                    1                    2   

                                                       
Actual label               intent_provide_flight_info  
intent_flight_search                                0  
intent_flight_status                                0  
intent_provide_flight_info                          5

In [14]:
ans = logreg.predict(vectorizer.transform(["can you help me to find a flight ?","we are leaving from San Francisco","how about the status of our flight departing from next Friday)"]))
print("prediction:\n {}".format(ans))

prediction:
 ['intent_flight_search' 'intent_provide_flight_info'
 'intent_flight_status']


In [15]:
print(classification_report(y_test, pred_logreg, target_names=sorted_labels))

                            precision    recall  f1-score   support

      intent_flight_search       0.90      1.00      0.95         9
      intent_flight_status       0.78      1.00      0.88         7
intent_provide_flight_info       1.00      0.62      0.77         8

                  accuracy                           0.88        24
                 macro avg       0.89      0.88      0.86        24
              weighted avg       0.90      0.88      0.87        24



In [16]:
scores = cross_val_score(SGDClassifier(loss="hinge", penalty="l2", max_iter=1000),vectorizer.transform(data.statement),data.label,cv=5)
print("Mean cross validation accuracy: {:.2f}".format(np.mean(scores)))

Mean cross validation accuracy: 0.87


In [17]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
clf.fit(x_train,y_train)
print("Training set score: {:.3f}".format(clf.score(x_train,y_train)))
print("Testing set score: {:.3f}".format(clf.score(x_test,y_test)))

Training set score: 1.000
Testing set score: 0.917


In [18]:
pred_sgd = clf.predict(x_test)
confusion_sgd = confusion_matrix(y_test,pred_sgd)
print("Confusion matrix:\n")

header = pd.MultiIndex.from_product([['Predicted label'],
                                     sorted_labels],names=['','Actual label'])
confusion_mtrix_df =  pd.DataFrame(confusion_sgd,index=sorted_labels, 
                  columns=header)
confusion_mtrix_df

Confusion matrix:



Predicted label                       \
Actual label               intent_flight_search intent_flight_status   
intent_flight_search                          8                    0   
intent_flight_status                          0                    7   
intent_provide_flight_info                    1                    0   

                                                       
Actual label               intent_provide_flight_info  
intent_flight_search                                1  
intent_flight_status                                0  
intent_provide_flight_info                          7

In [19]:
print(classification_report(y_test, pred_sgd, target_names=sorted_labels))

                            precision    recall  f1-score   support

      intent_flight_search       0.89      0.89      0.89         9
      intent_flight_status       1.00      1.00      1.00         7
intent_provide_flight_info       0.88      0.88      0.88         8

                  accuracy                           0.92        24
                 macro avg       0.92      0.92      0.92        24
              weighted avg       0.92      0.92      0.92        24



In [20]:
scores = cross_val_score(MLPClassifier(random_state=1, max_iter=1000),vectorizer.transform(data.statement),data.label,cv=5)
print("Mean cross validation accuracy: {:.2f}".format(np.mean(scores)))

Mean cross validation accuracy: 0.87


In [21]:
mlp = MLPClassifier(random_state=1, max_iter=500)
mlp.fit(x_train,y_train)
print("Training set score: {:.3f}".format(mlp.score(x_train,y_train)))
print("Testing set score: {:.3f}".format(mlp.score(x_test,y_test)))


pred_mlp = mlp.predict(x_test)
confusion_mlp = confusion_matrix(y_test,pred_mlp)
print("Confusion matrix:\n")

header = pd.MultiIndex.from_product([['Predicted label'],
                                     sorted_labels],names=['','Actual label'])
confusion_mtrix_df =  pd.DataFrame(confusion_mlp,index=sorted_labels, 
                  columns=header)
confusion_mtrix_df

Training set score: 1.000
Testing set score: 0.833
Confusion matrix:



Predicted label                       \
Actual label               intent_flight_search intent_flight_status   
intent_flight_search                          9                    0   
intent_flight_status                          0                    6   
intent_provide_flight_info                    1                    2   

                                                       
Actual label               intent_provide_flight_info  
intent_flight_search                                0  
intent_flight_status                                1  
intent_provide_flight_info                          5

In [22]:
print(classification_report(y_test, pred_mlp, target_names=sorted_labels))

                            precision    recall  f1-score   support

      intent_flight_search       0.90      1.00      0.95         9
      intent_flight_status       0.75      0.86      0.80         7
intent_provide_flight_info       0.83      0.62      0.71         8

                  accuracy                           0.83        24
                 macro avg       0.83      0.83      0.82        24
              weighted avg       0.83      0.83      0.83        24

